In [169]:
from simpletransformers.question_answering import QuestionAnsweringModel
from nested_lookup import nested_lookup
import pandas as pd
import numpy as np
import json
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download("punkt")
import string
import shutil
import datetime as dt
import torch
import random
import itertools
import sys, os
import warnings
warnings.filterwarnings('ignore')
from glob import glob
pd.options.display.max_rows = 999

[nltk_data] Downloading package stopwords to /home/ilker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ilker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
with open('train-v0.1.json' , encoding='utf-8') as json_file:
    data = json.load(json_file)
    
data_list = list()
for i in data['data']:
    for j in i['paragraphs']:
        for k in j['qas']:
            k['is_impossible'] = False
            for z in k['answers']:
                tmp = z['answer_start']
                z['answer_start'] = int(tmp)
        data_list.append(j)

In [67]:
model_name = "dbmdz/bert-base-turkish-cased"
model_base = model_name.split("/")[1].split("-")[0]
epoch = 3
batch_size = 16

dirs = glob("*")
dirs = pd.Series(dirs)
dels = dirs[dirs.isin(["cache_dir","outputs","runs"])].to_list()

for folder_name in dels:
    try:
        shutil.rmtree(folder_name)
    except:
        pass
        
model = QuestionAnsweringModel(model_base,
                               model_name, 
                               args = {"num_train_epochs" : epoch, "train_batch_size" : batch_size},
                               use_cuda = True)
model.train_model(data_list)

convert squad examples to features:   0%|          | 0/8308 [00:00<?, ?it/s]Could not find answer: '' vs. 'Yahudi'
Could not find answer: '' vs. 'Salisik asit, krizarobin ve iyodun, öğrencisi Hulusi Behçet ile birlikte de süblimenin'
add example index and unique id: 100%|██████████| 8308/8308 [00:00<00:00, 800530.17it/s]


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 4.943995Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 4.692345Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 3.027156Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.266419


Running loss: 0.236061


Running loss: 1.111119



## predict tweets

In [69]:
def soru_sor(soru,id, context,model):
    soru = [{
        'context': context,

        'qas': [
            {
                'question':soru,
                'id':id
            }
        ]
    }]
    
    return model.predict(soru)

In [70]:
def prepare_tweets(tweet_path, base_tt):

    f = open(tweet_path, "r", encoding="utf-8")
    a = f.read()
    a = pd.Series(a.split("\n"))
    tweets = a[a!=""].str.lower().to_list()

    base_questions = ["'a ne oldu", "'e ne oldu"
                      " ne zaman", " nerede ", " neden", " ne için",
                      " kimdir" , " kim" ]
    questions = (base_tt + pd.Series(base_questions)).to_list()

    question_list = []
    context_list = []
    id_list = []
    for tweet in tweets:
        for q in questions:

            question_list.append(q)
            context_list.append(tweet)

    for i in range(1, len(question_list) +1):
        id_list.append(str(i).zfill(4))
        
    return context_list, question_list, id_list

In [ ]:
def get_answers(tweet_path, base_tt):
    context_list, question_list, id_list = prepare_tweets(tweet_path, base_tt)

    predicted_answers = []

    for i in range(len(context_list)):
        answerNprobs = (soru_sor(question_list[i], id_list[i], context_list[i], model))
        answer = answerNprobs[0][0]["answer"][0]
        predicted_answers.append(answer)
    pd.DataFrame(predicted_answers).to_csv(base_tt + "_answers.csv")    

In [71]:
%%capture

tweet_path = "tweets/Demetevgar.txt"
base_tt = "demet evgar"
get_answers(tweet_path, base_tt)

tweet_path = "tweets/serdarhocanınyanındayız.txt"
base_tt = "serdar hocanın yanındayız"
get_answers(tweet_path, base_tt)

tweet_path = "tweets/YKSyierteleyin.txt"
base_tt = "YKSyi erteleyin"
get_answers(tweet_path, base_tt)

tweet_path = "tweets/kıdemtazminatımadokunma.txt"
base_tt = "kıdem tazminatıma dokunma"
get_answers(tweet_path, base_tt)

In [93]:
%%capture


context_list, question_list, id_list = prepare_tweets(tweet_path, base_tt)

predicted_answers = []

for i in range(len(context_list)):
    answerNprobs = (soru_sor(question_list[i], id_list[i], context_list[i], model))
    answer = answerNprobs[0][0]["answer"][0]
    predicted_answers.append(answer)
pd.DataFrame(predicted_answers).to_csv(base_tt + "_answers.csv")    

## analyze results

In [161]:
def remove_stopwords(some_list):
    for token in some_list:
        if token in stopwords.words("turkish"):
            del some_list[some_list.index(token)]
    return some_list

In [165]:
def frequent_words(tweet_path, base_tt):

    df = pd.read_csv(base_tt + "_answers.csv")
    tt = tweet_path.split("/")[1].split(".")[0]

    words = []
    for t in df["0"].str.split(" ", expand = True).astype(str).values:
        for k in t:
            if (k != "nan")  & (k != tt) & (k != "None") & (k not in tt) & ("#" not in k):
                words.append(k)

    freq = pd.Series(remove_stopwords(words)).value_counts()
    return freq

* 2020 yılında demet evgar'a arı ile ilgili bir şey olmuş, hastanedeymiş ve 40 yaşındaymış
* serdarhoca'nın kuzusu varmış :D turizmle ilgili bir şey olmuş ve seviyorlarmış 
* YKSyierteleyin: sınav olduğu belli öğrencileri ilgilendiriyor ikamet ile ilgili bir problem varmış
* işçilerle ilgili emekliliği ilgilendiriyor tamamlayıcı emeklilik umut gelecek keywordler
* metrik olarak şu seçilebilir cevaplardaki kelime sayısı/tt'lerdeki kelime sayısı: confidence gibi bir şey olabilir belki tam emin olamadım
* soruların ve cevapların sırası belli bu şekilde de anlam çıkarabiliriz most frequent az önce aklıma gelen bir yöntem. mesela hangi sorusuna en çok gelen yanıt şu ne zaman sorusuna bu. bu ve şu ile cümle kurdurabiliriz gibi

In [172]:
tweet_path = "tweets/Demetevgar.txt"
base_tt = "demet evgar"
freq = frequent_words(tweet_path, base_tt)
print(freq.sum())
freq.head(10)

230


2020         15
arı          14
sana          8
hayat         7
yaşında       7
40            7
bana          5
yapmaz        5
demet         5
hastanede     5
dtype: int64

In [173]:
tweet_path = "tweets/serdarhocanınyanındayız.txt"
base_tt = "serdar hocanın yanındayız"
freq = frequent_words(tweet_path, base_tt)
print(freq.sum())
freq.head(10)

230


serdarhocayanındayız    47
sonuna                   7
kadar                    7
gelen                    5
turizimden               5
kuzuların                5
sizi                     5
düşünüyor,               5
seviyor                  5
halkı                    5
dtype: int64

In [174]:
tweet_path = "tweets/YKSyierteleyin.txt"
base_tt = "YKSyi erteleyin"
freq = frequent_words(tweet_path, base_tt)
print(freq.sum())
freq.head(10)

364


sınav         14
ikamet        11
yakın         11
sınavı        11
öğrencinin    10
bir            8
yks            8
olsun          8
35.000         7
empty          7
dtype: int64

In [175]:
tweet_path = "tweets/kıdemtazminatımadokunma.txt"
base_tt = "kıdem tazminatıma dokunma"
freq = frequent_words(tweet_path, base_tt)
print(freq.sum())
freq.head(10)

460


işçinin          35
emeklilik        10
bir               8
tamamlayıcı       8
ben               8
işten             7
umudu             7
geleceği          7
alın              7
kolaylastirma     7
dtype: int64